# Multi-Armed Bandit Demo

This notebook demonstrates the usage of various bandit policies implemented in `research-bandits-methods`.

We'll simulate a simple K-armed bandit problem and compare different policies:
- ε-Greedy
- UCB (Upper Confidence Bound)
- Gaussian Thompson Sampling

## Setup

In [ ]:
import numpy as np

from research_bandits_methods.policies.markovian_policies import (
    EpsilonGreedy,
    GaussianThompson,
    UCB,
)

## Problem Setup: 5-Armed Bandit

We'll create a simple bandit environment with 5 arms, each having different mean rewards.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
rng = np.random.default_rng(42)

# Bandit setup
K = 5  # Number of arms
R = 10  # Number of parallel runs for Monte Carlo
T = 100  # Number of time steps

# True mean rewards for each arm (unknown to the policies)
true_means = np.array([0.3, 0.5, 0.8, 0.2, 0.6])
optimal_arm = np.argmax(true_means)
optimal_reward = true_means[optimal_arm]

print(f"True mean rewards: {true_means}")
print(f"Optimal arm: {optimal_arm} (mean reward: {optimal_reward:.2f})")

## Simulate Policies

In [ ]:
def simulate_bandit(policy, true_means, T, rng):
    """Simulate a bandit policy.

    Parameters
    ----------
    policy : MarkovianCumPullsCumRewardsPolicy
        The policy to simulate.
    true_means : np.ndarray
        True mean rewards for each arm.
    T : int
        Number of time steps.
    rng : np.random.Generator
        Random number generator.

    Returns
    -------
    rewards : np.ndarray
        Rewards obtained at each time step, shape (T, R).
    arms : np.ndarray
        Arms selected at each time step, shape (T, R).
    """
    R = policy.R
    rewards = np.zeros((T, R))
    arms = np.zeros((T, R), dtype=int)

    for t in range(1, T + 1):
        # Select arm
        arm = policy.select_arm(t)
        arms[t - 1] = arm

        # Generate rewards (Gaussian with unit variance)
        reward = rng.normal(true_means[arm], 1.0)
        rewards[t - 1] = reward

        # Update policy
        policy.update(arm, reward)

    return rewards, arms

### ε-Greedy Policy

In [ ]:
epsilon_greedy = EpsilonGreedy(K=K, R=R, epsilon=0.1, rng=rng)
rewards_eg, arms_eg = simulate_bandit(epsilon_greedy, true_means, T, rng)

# Calculate cumulative regret (averaged across runs)
cumulative_reward_eg = np.cumsum(rewards_eg, axis=0).mean(axis=1)
optimal_cumulative = np.arange(1, T + 1) * optimal_reward
regret_eg = optimal_cumulative - cumulative_reward_eg

print(f"\nε-Greedy Policy (ε=0.1):")
print(f"Final arm pull counts (averaged): {epsilon_greedy.counts.mean(axis=1)}")
print(f"Estimated means: {epsilon_greedy.means.mean(axis=1)}")
print(f"Final cumulative regret: {regret_eg[-1]:.2f}")

### UCB Policy

In [ ]:
ucb = UCB(K=K, R=R, c=2.0)
rewards_ucb, arms_ucb = simulate_bandit(ucb, true_means, T, rng)

cumulative_reward_ucb = np.cumsum(rewards_ucb, axis=0).mean(axis=1)
regret_ucb = optimal_cumulative - cumulative_reward_ucb

print(f"\nUCB Policy (c=2.0):")
print(f"Final arm pull counts (averaged): {ucb.counts.mean(axis=1)}")
print(f"Estimated means: {ucb.means.mean(axis=1)}")
print(f"Final cumulative regret: {regret_ucb[-1]:.2f}")

### Gaussian Thompson Sampling

In [ ]:
thompson = GaussianThompson(K=K, R=R, prior_mean=0.5, prior_var=1.0, rng=rng)
rewards_ts, arms_ts = simulate_bandit(thompson, true_means, T, rng)

cumulative_reward_ts = np.cumsum(rewards_ts, axis=0).mean(axis=1)
regret_ts = optimal_cumulative - cumulative_reward_ts

print(f"\nGaussian Thompson Sampling:")
print(f"Final arm pull counts (averaged): {thompson.counts.mean(axis=1)}")
print(f"Estimated means: {thompson.means.mean(axis=1)}")
print(f"Final cumulative regret: {regret_ts[-1]:.2f}")

## Summary

All three policies successfully learn to identify the optimal arm (arm 2 with mean 0.8) over time.

Key observations:
- **ε-Greedy**: Simple and effective, but continues to explore uniformly even after finding the best arm
- **UCB**: Balances exploration and exploitation using confidence bounds
- **Thompson Sampling**: Bayesian approach that naturally balances exploration and exploitation

The policies can be compared by examining:
1. Final arm pull counts (how many times each arm was selected)
2. Estimated means (how well the policy learned the true means)
3. Cumulative regret (total opportunity cost from not always selecting the optimal arm)